need a process to:
1) find all games and game ids
2) find all players and player ids
3) parse box scores for a game id


to find game ids:

from nba_api.stats.endpoints import leaguegamefinder
# Query for games where the Celtics were playing
#gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
gamefinder = leaguegamefinder.LeagueGameFinder()
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.head()


to find a box score from a game id:
from nba_api.stats.endpoints import boxscoreadvancedv2
box_scores = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id='0021800283')
box_scores.get_data_frames()[0].head()

to find the score of a game:
from nba_api.stats.endpoints import boxscoresummaryv2

In [32]:
import pandas as pd
from databaseClass import DB
import utils
import sql_files
import pandas as pd
import numpy as np
import time

userName = utils.userName
userPass = utils.userPass
dbName = utils.dbName
db = DB(userName=userName, userPass=userPass, dataBaseName=dbName)

In [25]:
def generate_sql(input_sql_file):
    with open(input_sql_file, 'r') as file:
        sql = file.read()
        return sql
    
def create_or_replace_table(sql_file, table_name):
    '''postgres does not have a create or replace table option.  so we do that here instead'''
    try:
        db.BuildTableFromQuery(generate_sql(sql_file), table_name)
        print(f"{table_name} didn't exist. completed loading")
    except:
        db.dropTable(table_name)
        db.BuildTableFromQuery(generate_sql(sql_file), table_name)
        print(
            f"{table_name} did exist. dropped old version and completed loading new one")
        
def get_games_from_team_id(team_id):
    from nba_api.stats.endpoints import leaguegamefinder
    # Query for games where the team was playing
    gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=team_id)
    # The first DataFrame of those returned is what we want.
    games = gamefinder.get_data_frames()[0]
    return games

1) find all games and game ids

In [4]:
from nba_api.stats.endpoints import leaguegamefinder
# Query for games where the Celtics were playing
#gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable=celtics_id)
gamefinder = leaguegamefinder.LeagueGameFinder()
# The first DataFrame of those returned is what we want.
games = gamefinder.get_data_frames()[0]
games.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,42022,1611661329,CHI,Chicago Sky,1042200212,2022-08-31,CHI vs. CON,W,200,85,...,0.682,3,26,29,22,7,8,12,15,10.0
1,42022,1611661319,LVA,Las Vegas Aces,1042200202,2022-08-31,LVA vs. SEA,None,100,36,...,0.750,2,15,17,6,0,3,4,6,0.0
2,42022,1611661323,CON,Connecticut Sun,1042200212,2022-08-31,CON @ CHI,L,201,77,...,0.500,7,32,39,19,7,3,12,17,-8.4
3,42022,1611661328,SEA,Seattle Storm,1042200202,2022-08-31,SEA @ LVA,None,100,36,...,1.000,1,18,19,8,0,4,2,8,0.0
4,42022,1611661323,CON,Connecticut Sun,1042200211,2022-08-28,CON @ CHI,W,200,68,...,0.818,12,35,47,23,7,3,9,7,5.0


In [10]:
#send the games dataframe to the games table
db.DFtoDB(games, "games")

In [22]:
# generate table of all teams
create_or_replace_table('sql_files/makeTable_Teams_Meta_Data.sql', 'Teams_Meta_Data')

Teams_Meta_Data didn't exist. completed loading


In [34]:
# generate list of all games (the initial games table is not comprehensive because i was capped at 30,000)
#1 get a list of all team_ids from the Teams_Meta_Data table, for each team_id, get all the games.
#2 in order to get all games, run get_games_from_team_id
#3 send dataframe to games table

#1
def generate_complete_games_table():
    df_teams = db.DBtoDF(generate_sql('sql_files/query_Generate_Team_IDs.sql'))
    teams_list = df_teams['TEAM_ID']

    #2 & 3

    for team in teams_list:
        df_games_for_a_team = get_games_from_team_id(team)
        try:
            db.DFtoDB(df_games_for_a_team, "games_loading")
            print(f"completed: {team}")
        except:
            print(f"failed: {team}")
        time.sleep(10)
        
generate_complete_games_table()

# note there are a few teams that had irregular field types, and couldn't be loaded into our regular table.  
# i guess this needs to be handled manually.  maybe from staging to final table, i need to change field types. 

# the bad ids are below
# 1610612762
# 1610612755
# 1610612744
# 1610612758
# 1610612737


completed: 1612709908
completed: 1611661322
completed: 1610612766
completed: 1612709923
completed: 1621980020
completed: 1612709924
completed: 1610612740
completed: 1612709902
completed: 1621980012
completed: 1621980034
completed: 1612709900
completed: 1612709905
completed: 215
completed: 1610616843
completed: 1610616848
completed: 1611665410
completed: 1612709907
completed: 270
completed: 1879048193
completed: 1621980030
completed: 1612709927
completed: 228
completed: 1621980032
completed: 1612709918
completed: 1612709921
completed: 1610612752
completed: 1612709906
failed: 1610612762
completed: 1609800002
completed: 273
completed: 229
completed: 12329
completed: 272
completed: 12325
completed: 1621980018
completed: 70
completed: 15018
completed: 15021
completed: 1621980007
completed: 1609800004
completed: 278
completed: 1612709932
completed: 1610612753
completed: 84
completed: 276
completed: 1610612765
completed: 1611661319
completed: 1621980019
completed: 112
completed: 271
completed

In [55]:
# check all the game_ids games table.  compare this to all the game_ids in the boxscore table.  
# get list of missing ids from boxscore table. 
# run through this list, generating new boxscores


#to find a box score from a game id:
from nba_api.stats.endpoints import boxscoreadvancedv2
box_scores = boxscoreadvancedv2.BoxScoreAdvancedV2(game_id='0021800283')
box_scores.get_data_frames()[0].head()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,...,TM_TOV_PCT,EFG_PCT,TS_PCT,USG_PCT,E_USG_PCT,E_PACE,PACE,PACE_PER40,POSS,PIE
0,0021800283,1610612738,BOS,Boston,1628369,Jayson Tatum,Jayson,F,,36:24,...,10.5,0.750,0.750,0.186,0.192,101.33,100.22,83.52,76,0.149
1,0021800283,1610612738,BOS,Boston,201143,Al Horford,Al,F,,31:22,...,14.3,0.364,0.364,0.171,0.177,101.27,101.76,84.80,67,0.041
2,0021800283,1610612738,BOS,Boston,202694,Marcus Morris Sr.,Marcus,C,,29:38,...,0.0,0.650,0.650,0.143,0.145,102.79,102.05,85.04,62,0.095
3,0021800283,1610612738,BOS,Boston,1627759,Jaylen Brown,Jaylen,G,,30:24,...,16.7,0.778,0.810,0.164,0.168,102.38,101.84,84.87,64,0.110
4,0021800283,1610612738,BOS,Boston,202681,Kyrie Irving,Kyrie,G,,34:33,...,14.3,0.375,0.389,0.345,0.356,102.79,101.40,84.50,74,0.066
